# Classifying digits using a fully connected neural network

In this practical exercice we will build a fully convolutional neural network using keras. We will train it to classify image digits from the MNIST database.

Based on https://raw.githubusercontent.com/keras-team/keras/master/examples/mnist_mlp.py

In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import RMSprop
import numpy as np
import matplotlib.pyplot as plt

# Magic used by the notebook to show figures inline
%matplotlib inline
# matplotlib default values
plt.rcParams['figure.figsize'] = (10.0, 8.0)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# auto-reloading packages
%load_ext autoreload
%autoreload 2



In [ ]:
batch_size = 128
epochs = 20
val_nb = 5000  # number of validation samples

In [ ]:
# Load and have a look at the data
(x, y), (x_test_ori, y_test_ori) = mnist.load_data()

# Visualize a single digit, with its class
index = 2
plt.imshow(x[index])
print("Class: ", y[index])

In [ ]:
# Data management

nb_samples = x.shape[0]

if val_nb > nb_samples:
    raise ValueError("You need some samples to train your network!")

x = x.reshape(nb_samples, 784)
x_test = x_test_ori.reshape(x_test_ori.shape[0], 784)
x = x.astype('float32')
x_test = x_test.astype('float32')
x /= 255
x_test /= 255

x_val = x[:val_nb, ]
x_train = x[val_nb:, ]
y_val = y[:val_nb]
y_train = y[val_nb:]

print(x_train.shape, 'x train samples')
print(x_val.shape, 'x val samples')
print(x_test.shape, 'x test samples')
print(y_train.shape, 'y train samples')
print(y_val.shape, 'y val samples')
print(y_test_ori.shape, 'y test samples')

# convert class vectors to binary class matrices
num_classes = max(y) + 1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test_ori, num_classes)


# Model definition

The following model uses keras to build a fully convolutional network. It has to respect some constraints:

- The input shape has to match the size of each input sample. 
- The ouptput should be of size 10 (num_classes)

Other than that, you change the number of layers as well as the size of each of them.

In [ ]:
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(x_train.shape[1],)))
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.summary()


# Model training

The following model takes care of training.

Firstly, the model has to be 'compiled'. This operations lets the user choose the loss, the optmizer and the metrics, then configures the model for training.

Secondly, the 'fit' method run the optmization. Traning and validation data are specified here, as well as batch size and the number of epochs.

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

output = model.fit(x_train, y_train,
                   batch_size=batch_size,
                   epochs=epochs,
                   validation_data=(x_val, y_val),
)


# Analysis of the results

Visualizing what is going on is extremely important. For that:

- inspecting traning and validation performance is essential;

- looking at the errors might also be interesting.

Is there overfitting? How can it be reduced?
Is the network 'confident' when making errors?


In [ ]:
plt.plot(output.epoch, output.history['loss'], label='train')
plt.plot(output.epoch, output.history['val_loss'], label='val')
plt.title('Training and validation performance')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend()
# plt.ylim(0.2, 0.8)

In [ ]:
y_predict_proba = model.predict(x_test)
# print("Prediction example:", y_predict_proba[0])

y_predict = np.argmax(y_predict_proba, 1)
# print(y_predict[3000:3010])

diff = y_test_ori != y_predict
# print("Difference mask: ", diff)
x_test_errors = x_test_ori[diff]
y_test_errors = y_test_ori[diff]
y_predict_errors = y_predict[diff]
y_predict_proba_errors = y_predict_proba[diff]

index = 0
print("Correct label is: ", y_test_errors[index])
print("Predicted label is: ", y_predict_errors[index])
print("Probabilities: ", y_predict_proba_errors[index])
plt.imshow(x_test_errors[index])

# Testing

Testing is the last stage of the learning process. Good practice recommends to do it only once, when you have completely finised with the optimization of the network parameters and hyperparameters.

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])